# Selenium 네이버 뉴스 댓글 스크래핑

## Selenium 및 웹 드라이버 설치

In [ ]:
!pip install Selenium
!apt-get upgrade
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 2.9MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Calculating upgrade... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following packages have been kept back:
  libcudnn7 libcudnn7-dev libnccl-dev libnccl2
The following packages will be upgraded:
  base-files binutils binutils-common binutils-x86-64-linux-gnu bsdutils
  cuda-compat-10-1 e2fsprogs fdisk libbinutils libblkid1 libc-bin libcom-err2
  libcublas-dev libcublas10 libext2fs2 libfdisk1 libgcrypt20 libgnutls30
  libldap-2.4-2 libldap-common libmount1 libpam-modules libpam-modules-bin
  libpam-runtime libpam0g libsasl2-2 libsasl2-modules-db libseccomp2
  libsmartcols1 libsqlite3-0 libss2 linux-libc-dev mount openssl util-linux
35 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Need to get 100 MB of archives.
After this ope

## 라이브러리 import

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

chorme_options = webdriver.ChromeOptions()
chorme_options.add_argument('--headless')
chorme_options.add_argument('--no-sandbox')
chorme_options.add_argument('--disable-dev-shm-usage')

## `news_scraping()`: 뉴스 스크래핑

In [ ]:
def news_scraping(news_url, wd):

  press = wd.find_element_by_xpath('//*[@id="main_content"]/div[1]/div[1]/a/img').get_attribute('title')
  title = wd.find_element_by_id('articleTitle').text
  datetime = wd.find_element_by_class_name('t11').text
  article = wd.find_element_by_id('articleBodyContents').text
  article = article.replace("// flash 오류를 우회하기 위한 함수 추가", "")
  article = article.replace("function _flash_removeCallback() {}", "")  
  article = article.replace("\n", "")  
  article = article.replace("\t", "")

  good = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[1]/a/span[2]').text
  warm = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[2]/a/span[2]').text
  sad = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[3]/a/span[2]').text
  angry = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[4]/a/span[2]').text
  want = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[5]/a/span[2]').text
  recom = wd.find_element_by_xpath('//*[@id="toMainContainer"]/a/em[2]').text

  print("뉴스: ", [title, press, datetime, article, good, warm, sad, angry, want, recom, news_url])

  return [title, press, datetime, article, good, warm, sad, angry, want, recom, news_url]

## `comments_scraping()`: 뉴스 댓글 스크래핑

In [ ]:
def comments_scraping(news_url, wd):
  
  try:
    button_view = wd.find_element_by_class_name('u_cbox_btn_view_comment').click()
    print("[댓글 더보기]", end="")
    time.sleep(1)

    while True:
      wd.find_element_by_class_name('u_cbox_btn_more').click()
      print("[더보기]", end="")
      time.sleep(1)
  except:
    pass

  btn_reply_list = wd.find_elements_by_class_name('u_cbox_btn_reply')
  for btn_reply in btn_reply_list:
    btn_reply.send_keys('\n') #클릭 기능
    print("[답글]", end="")
    time.sleep(1)

  print("[댓글 스크래핑]")
  comments_idx = 0
  comments_df = pd.DataFrame(columns=("Contents", "Name", "Datetime", "Recommend", "Unrecommend", "URL"))

  comments = wd.find_elements_by_class_name('u_cbox_comment_box')
  for comment in comments:
    try:
      name = comment.find_element_by_class_name('u_cbox_nick').text
      date = comment.find_element_by_class_name('u_cbox_date').text
      contents = comment.find_element_by_class_name('u_cbox_contents').text
      recomm = comment.find_element_by_class_name('u_cbox_cnt_recomm').text
      unrecomm = comment.find_element_by_class_name('u_cbox_cnt_unrecomm').text
      print(f" 댓글 #{comments_idx+1}:",[contents, name, date, recomm, unrecomm, news_url])
      comments_df.loc[comments_idx] = [contents, name, date, recomm, unrecomm, news_url]
      comments_idx += 1
    except NoSuchElementException:
      #print("[삭제되거나 부적절한 댓글]")
      continue

  return comments_df

## `scraping()`: 스크래핑 함수

In [ ]:
def scraping(news_max=3):
  wd = webdriver.Chrome('chromedriver', options=chorme_options)
  wd.implicitly_wait(3)

  wd.execute_script('window.open("about:blank", "_blank");')
  tabs = wd.window_handles

  wd.switch_to.window(tabs[0])
  query = input("검색어 입력: ")
  search_url = "https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query=" + query
  wd.get(search_url)

  news_idx = 0
  news_df = pd.DataFrame(columns=("Title", "Press", "DateTime", "Article", "Good", "Warm", "Sad", "Angry", "Want", "Recommend", "URL"))
  comments_df = pd.DataFrame()

  while True:
    inline_list = wd.find_elements_by_class_name('txt_inline')
    for inline in inline_list:
      try:
        news_url = inline.find_element_by_class_name('_sp_each_url').get_attribute('href')
      except:
        continue
      
      wd.switch_to.window(tabs[1])
      wd.get(news_url)

      news_df.loc[news_idx] = news_scraping(news_url, wd) #함수 호출
      news_idx += 1

      df = comments_scraping(news_url, wd) #함수 호출
      comments_df = pd.concat([comments_df, df]) #데이터프레임 하나로 합침

      if news_idx >= news_max:
        break

    if news_idx >= news_max:
        break

    try:
      wd.switch_to.window(tabs[0])
      wd.find_element_by_class_name('next').click()
      time.sleep(1)
    except:
      break

  wd.close()

  return news_df, comments_df

## 스크래핑 실행

In [ ]:
news_df, comments_df = scraping()

검색어 입력: 삼성바이오로직스
뉴스:  ['[단독] 삼바·SK바이오팜이 활용한 상장 특례, 이젠 어려워진다', '조선비즈', '2020.08.27. 오전 6:03', '거래소, 시총·자기자본만 보던 상장규정 심사 강화한국거래소가 당기순이익, 영업이익, 매출액 등 재무현황과 관계없이 시가총액과 자기자본만을 보고 상장을 허용해주던 상장규정을 개정하기로 했다. 지금까지 거래소는 바이오 등 미래 성장성이 높은 기업에 대해선 시가총액 6000억원 이상, 자기자본 2000억원 이상이면 상장주관사(IB)의 실사에서 큰 문제점이 발견되지 않는 경우 유가증권시장 상장을 허용했다. 그러나 앞으로는 시가총액, 자기자본 기준을 충족하더라도 상장 후 성장가능성을 살펴보기 위해 자금사용계획서, 사업전략 등을 꼼꼼하게 살펴본 후에 상장을 허용해주기로 했다.시가총액 6000억원 이상, 자기자본 2000억원 이상이면 매출액이나 영업이익, 순이익 등 재무성과를 보지 않고 유가증권시장에 상장을 허용해준 규정은 2015년 11월에 도입됐는데 그 동안 혜택을 본 기업은 삼성바이오로직스(207940)(2016년 11월 상장), SK바이오팜(2020년 7월) 단 2곳뿐이다. 이 규정이 도입된 직후 수년간 당기순손실을 기록했던 삼성바이오로직스가 유가증권시장에 입성해 삼성바이오로직스를 위한 특혜규정이 아니냐는 논란도 제기됐다.지난달 2일 오전 서울 여의도 한국거래소에서 열린 SK바이오팜 코스피 신규상장 기념식에서 조정우 SK바이오팜 대표(왼쪽)가 정지원 한국거래소 이사장에게 상장계약서를 전달받고 있다. / 연합뉴스27일 금융투자업계에 따르면 한국거래소는 유가증권시장 상장규정 시행세칙을 개정하는 작업을 진행하고 있다. 개정된 세칙은 다음달 말쯤 시행될 계획이다. 거래소가 개정하는 것은 ‘시가총액 6000억원 이상, 자기자본 2000억원 이상’이면 유가증권시장에 상장할 수 있는 상장규정으로 ‘대형성장유망기업 요건’으로 불리는 조항이다.거래소는 앞으로 상장주관사(IB)가 기업의 공모자금 사용계획, 사업계획 달

In [ ]:
news_df

,Title,Press,DateTime,Article,Good,Warm,Sad,Angry,Want,Recommend,URL
0,"[단독] 삼바·SK바이오팜이 활용한 상장 특례, 이젠 어려워진다",조선비즈,2020.08.27. 오전 6:03,"거래소, 시총·자기자본만 보던 상장규정 심사 강화한국거래소가 당기순이익, 영업이익,...",7,0,0,87,2,11,https://news.naver.com/main/read.nhn?mode=LSD&...
1,"삼성바이오로직스, 장중 시총 2위 등극…SK하이닉스 제쳐",한국경제,2020.08.20. 오전 9:36,삼성바이오로직스가 장중 시가총액 2위에 올라섰다. SK하이닉스가 주춤한 틈을 타 추...,0,0,0,4,0,,https://news.naver.com/main/read.nhn?mode=LSD&...
2,"삼성바이오로직스, 송도에 4공장 설립…""단일공장 세계 최대""(종합)",연합뉴스,2020.08.11. 오전 10:31,"1조7천억원 투자해 25만6천ℓ 규모 설립…완공 시 총 4개 공장 62만ℓ 확보""송...",105,0,1,23,1,16,https://news.naver.com/main/read.nhn?mode=LSD&...


In [ ]:
comments_df

,Contents,Name,Datetime,Recommend,Unrecommend,URL
0,별 개트집은...지금 시총 몇위인지 봐~이런 기업이 초기 투자성장기에 적자였다고 I...,sans****,2020.08.26. 21:53,126,51,https://news.naver.com/main/read.nhn?mode=LSD&...
1,개소리하네. 나스닥갔으면 파산에 죄롱이 50년은 감빵생활이다 알기나 하나? 글구 지...,ks95****,2020.08.26. 22:33,16,16,https://news.naver.com/main/read.nhn?mode=LSD&...
2,삼성일바들 ㅋㅋ 정말 삼성 정직 알바견들이 있다는 말이 맞구나. 이 말도 안되는 댓...,gjl0****,2020.08.26. 23:55,5,5,https://news.naver.com/main/read.nhn?mode=LSD&...
3,나스닥무새들아 거기는 아무거나 다 상장시켜주는줄아나봐?,crea****,2020.08.27. 03:28,0,0,https://news.naver.com/main/read.nhn?mode=LSD&...
4,나스닥으로 갈뻔했던거 그나마 간신히 잡은건데 그걸 특혜라고 표현하네. 삼성도 기가차...,whit****,2020.08.26. 22:00,98,39,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...,...
57,삼성 이제 그만좀괴롭혀라 이 운동권무능독재정권아,wnfl****,2020.08.11. 02:50,0,4,https://news.naver.com/main/read.nhn?mode=LSD&...
58,이재용이는 삼성바이오로직스 천문학적 분식회계와 주가조작등으로 무기징역에 처해라...,navy****,2020.08.11. 02:27,2,7,https://news.naver.com/main/read.nhn?mode=LSD&...
59,근데 물통 공장에 뭐하는데요?? 기술력도없고 그냥 공장만 세우고 나머지 공장들은 돌...,bogg****,2020.08.11. 01:48,4,12,https://news.naver.com/main/read.nhn?mode=LSD&...
60,너 같으면 노트북 숨기려고 1조 넘게 돈 쓰냐? ㅋ 뭔가 니 말이 핵심을 팍 찌르는...,kvag****,2020.08.11. 02:00,1,0,https://news.naver.com/main/read.nhn?mode=LSD&...
